In [22]:
import os

import boto3
import botocore
import configparser

from botocore.exceptions import ClientError

In [23]:
config_aws = configparser.ConfigParser()
config_aws.read_file(open('../config/aws_credentials.cfg'))

KEY                    = config_aws.get('AWS','KEY')
SECRET                 = config_aws.get('AWS','SECRET')
IAM_ARN                = config_aws.get('AWS', 'IAM_ARN')

In [66]:
config_instance = configparser.ConfigParser()
config_instance.read_file(open('../config/aws_setup.cfg'))

AMI_ID                 = config_instance.get("AWS", "AMI_ID")
KEY_PAIR_NAME          = config_instance.get("AWS", "KEY_PAIR_NAME")
## EC2 config
EC2_REGION             = config_instance.get("EC2", "REGION")
EC2_TYPE               = config_instance.get("EC2", "INSTANCE_TYPE")
EC2_NAME               = config_instance.get("EC2", "NAME")
## VPC config
VPC_NAME               = config_instance.get("VPC", "NAME")
VPC_CIDR_BLOCK         = config_instance.get("VPC", "CIDR_BLOCK")
## public subnet config
PUB_SUBNET_NAME        = config_instance.get("SUBNET", "PUB_SUBNET_NAME")
PUB_SUBNET_CIDR_BLOCK  = config_instance.get("SUBNET", "PUB_SUBNET_CIDR_BLOCK")
PUB_SUBNET_REGION      = config_instance.get("SUBNET", "PUB_SUBNET_REGION")
PVR_SUBNET_NAME        = config_instance.get("SUBNET", "PVR_SUBNET_NAME")
## public subnet for EMR cluster config
PUB_SPARK_SUBNET_NAME        = config_instance.get("SUBNET", "PUB_SPARK_SUBNET_NAME")
PUB_SPARK_SUBNET_CIDR_BLOCK  = config_instance.get("SUBNET", "PUB_SPARK_SUBNET_CIDR_BLOCK")
PUB_SPARK_SUBNET_REGION      = config_instance.get("SUBNET", "PUB_SPARK_SUBNET_REGION")
## private subnet (empty)
PVR_SUBNET_NAME        = config_instance.get("SUBNET", "PVR_SUBNET_NAME")
PVR_SUBNET_CIDR_BLOCK  = config_instance.get("SUBNET", "PVR_SUBNET_CIDR_BLOCK")
PVR_SUBNET_REGION      = config_instance.get("SUBNET", "PVR_SUBNET_REGION")
## private subnet for RDS
PVR_RDS_SUBNET_NAME        = config_instance.get("SUBNET", "PVR_RDS_SUBNET_NAME")
PVR_RDS_SUBNET_CIDR_BLOCK  = config_instance.get("SUBNET", "PVR_RDS_SUBNET_CIDR_BLOCK")
PVR_RDS_SUBNET_REGION      = config_instance.get("SUBNET", "PVR_RDS_SUBNET_REGION")
## route table config
RT_NAME                = config_instance.get("ROUTE_TABLE", "NAME")
## internet gateway config
IG_NAME                = config_instance.get("IG", "NAME")
## security group
RDS_SG_NAME            = config_instance.get("SG", 'RDS_SG_NAM')
RDS_NAME               = config_instance.get("RDS", "NAME")
RDS_IDENTIFIER         = config_instance.get("RDS", "RDS_IDENTIFIER")


In [25]:
def get_keypair(ec2):
    # call the boto ec2 function to create a key pair
    key_pair = ec2.create_key_pair(KeyName=KEY_PAIR_NAME)
    print("Created a new key pair in AWS.")

    # capture the key and store it in a file
    KeyPairOut = str(key_pair.key_material)

    # create a file to store the key locally
    print("Saving the keypair.")
    key_pair_path = KEY_PAIR_NAME + '.pem'
    with open(key_pair_path, 'w') as f:
        f.write(KeyPairOut)
    os.chmod(key_pair_path, 0o600)
    print("Changed access permission to read-only.")

In [26]:
def create_vpc(ec2):
    # create a new VPC
    print("Creating VPC...")
    vpc = ec2.create_vpc(CidrBlock=VPC_CIDR_BLOCK,
                         TagSpecifications=[{'ResourceType': 'vpc',
                                             'Tags':[{"Key": "Name", 
                                                      "Value": VPC_NAME},
                                                    ]
                                            }])
    
    # wait till available and return VPC ID
    vpc.wait_until_available()
    print(f"VPC {VPC_NAME} is available!")
    return vpc

In [27]:
def create_subnet(ec2, vpc, 
                  subnet_name,
                  subnet_region, 
                  subnet_cidr_block,
                  subnet_type='private'):
    # create a public subnet within the VPC
    print("Creating a "+subnet_type+" subnet...")
    subnet = ec2.create_subnet(
        AvailabilityZone=subnet_region,
    #     AvailabilityZoneId='use2-az2',
        CidrBlock=subnet_cidr_block,
        VpcId=vpc.vpc_id,
        DryRun=False,
        TagSpecifications=[{
            'ResourceType':'subnet',
            'Tags':[{"Key": "Name", "Value": subnet_name},
                   ]
        }])
    print(f"Subnet {subnet_name} is available!")
    return subnet

In [28]:
def stop_instance(ec2_client, instances):
    # get a list of instance ids
    instances_ids = [i.instance_id for i in instances]
    # start the instances
    ec2_client.stop_instances(InstanceIds=instances_ids)
    
    # wait till instance is stopped
    waiter = ec2_client.get_waiter('instance_stopped')
    waiter.wait(InstanceIds=instances_ids)
    print("EC2 instance has stopped!")

In [29]:
def start_instance(ec2_client, instances):
    # get a list of instance ids
    instances_ids = [i.instance_id for i in instances]
    # start the instances
    ec2_client.start_instances(InstanceIds=instances_ids)
    
    # wait till instance is ready
    waiter = ec2_client.get_waiter('instance_running')
    waiter.wait(InstanceIds=instances_ids)
    print("EC2 instance is ready!")

In [30]:
## create a ec2 resource instance
ec2 = boto3.resource('ec2', 
                    region_name = EC2_REGION,
                    aws_access_key_id = KEY,
                    aws_secret_access_key = SECRET)

In [31]:
## create a ec2 client instnace
ec2_client = boto3.client('ec2')

In [32]:
rds_client = boto3.client('rds')

In [33]:
## create a key pair and download them 
get_keypair(ec2)

Created a new key pair in AWS.
Saving the keypair.
Changed access permission to read-only.


In [34]:
## create a VPC instance
vpc = create_vpc(ec2)

Creating VPC...
VPC IMMIGRATE_DEMOGRAPHICS_VPC is available!


In [35]:
# create a public subnet
subnet_pub_ec2 = create_subnet(ec2=ec2, vpc=vpc, subnet_name=PUB_SUBNET_NAME,
                        subnet_region=PUB_SUBNET_REGION, 
                        subnet_cidr_block=PUB_SUBNET_CIDR_BLOCK,
                        subnet_type='public')
# create a private subnet
subnet_prv_empty = create_subnet(ec2=ec2, vpc=vpc, subnet_name=PVR_SUBNET_NAME,
                        subnet_region=PVR_SUBNET_REGION, 
                        subnet_cidr_block=PVR_SUBNET_CIDR_BLOCK,
                        subnet_type='private')

Creating a public subnet...
Subnet IMMIGRATE_DEMOGRAPHICS_PUB is available!
Creating a private subnet...
Subnet IMMIGRATE_DEMOGRAPHICS_PVR is available!


In [36]:
## create a public subnet for EMR cluster
subnet_pub_emr = create_subnet(ec2=ec2, vpc=vpc, subnet_name=PUB_SPARK_SUBNET_NAME,
                        subnet_region=PUB_SPARK_SUBNET_REGION,
                        subnet_cidr_block=PUB_SPARK_SUBNET_CIDR_BLOCK,
                        subnet_type="public")
subnet_prv2_rds = create_subnet(ec2=ec2, vpc=vpc, subnet_name=PVR_RDS_SUBNET_NAME,
                        subnet_region=PVR_RDS_SUBNET_REGION,
                        subnet_cidr_block=PVR_RDS_SUBNET_CIDR_BLOCK,
                        subnet_type="private")

Creating a public subnet...
Subnet IMMIGRATE_DEMOGRAPHICS_SPARK_PUB is available!
Creating a private subnet...
Subnet IMMIGRATE_DEMOGRAPHICS_RDS_PVR is available!


In [37]:
## create internet gateway
ig = ec2.create_internet_gateway(TagSpecifications=[{
            'ResourceType':'internet-gateway',
            'Tags':[{"Key": "Name", "Value": IG_NAME},
                   ]}])
## attach the internet gateway to the VPC
vpc.attach_internet_gateway(InternetGatewayId = ig.id)

{'ResponseMetadata': {'RequestId': 'f2eab257-8b56-4e2b-a08c-b4a0bd88d01a',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'f2eab257-8b56-4e2b-a08c-b4a0bd88d01a',
   'content-type': 'text/xml;charset=UTF-8',
   'content-length': '243',
   'date': 'Tue, 06 Oct 2020 19:15:36 GMT',
   'server': 'AmazonEC2'},
  'RetryAttempts': 0}}

In [ ]:
###### the next 2 steps is to allow SSH connection from outside of VPC to the EC instance

In [38]:
## 1.1 create a new route table for public subnet and
routetable = vpc.create_route_table(
    TagSpecifications=[
        {
            'ResourceType': 'route-table',
            'Tags': [
                {
                    'Key': 'Name',
                    'Value': RT_NAME
                },
            ]
        },
    ]
)
## 1.2 create new route, that allows traffic outside of VPC to go to the internet gateway
route = routetable.create_route(DestinationCidrBlock='0.0.0.0/0', GatewayId=ig.id)
## 1.3 attach the new route to the public subnet
routetable.associate_with_subnet(SubnetId=subnet_pub_ec2.id)
routetable.associate_with_subnet(SubnetId=subnet_pub_emr.id)

ec2.RouteTableAssociation(id='rtbassoc-0bf9ac3c663047b74')

In [44]:
## 2.1 get default security group id
sg = ec2_client.describe_security_groups(Filters=[{'Name': 'vpc-id',
                                                    'Values': [vpc.vpc_id]}
                                                 ])

for sg_ in sg['SecurityGroups']:
    if sg_['Description'] == 'default VPC security group':
        sg_default = sg_
        break
        
sg_default_id = sg_default['GroupId']
## 2.2 add imbound rule for the security group, allowing SSH connect from all the internet
ec2_client.authorize_security_group_ingress(GroupId = sg_default_id,
                                        IpPermissions=[
                                        {
                                            'FromPort': 8998,
                                            'IpProtocol': 'tcp',
                                            'IpRanges': [
                                                {
                                                    'CidrIp': '0.0.0.0/0',
                                                    'Description': 'SSH access from outside',
                                                },
                                            ],
                                            'ToPort': 8998,
                                        },
                                        {
                                            "FromPort": 8080,
                                            "IpProtocol": "tcp",
                                            "IpRanges": [
                                                {
                                                    "CidrIp": "0.0.0.0/0",
                                                    "Description": "Apache Airflow",
                                                },
                                            ],
                                            "ToPort": 8080,
                                        }
                                    ],)

{'ResponseMetadata': {'RequestId': '47bdae6f-48fb-44c8-87bc-34018d7324ad',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '47bdae6f-48fb-44c8-87bc-34018d7324ad',
   'content-type': 'text/xml;charset=UTF-8',
   'content-length': '259',
   'date': 'Tue, 06 Oct 2020 19:31:07 GMT',
   'server': 'AmazonEC2'},
  'RetryAttempts': 0}}

In [48]:
## create EC2 instance
instances = ec2.create_instances(
     ImageId=AMI_ID,
     MinCount=1,
     MaxCount=1,
     InstanceType=EC2_TYPE,
     KeyName=KEY_PAIR_NAME,
     NetworkInterfaces=[{
             'DeviceIndex':0,
             'SubnetId': subnet_pub_ec2.id}],
#               'SubnetId': 'subnet-0ee2d46148e05870f'}],
     TagSpecifications=[{
            'ResourceType':'instance',
            'Tags':[{"Key": "Name", "Value": EC2_NAME}]
            }]
 )

# get instance ids
instances_ids = [i.instance_id for i in instances]

# wait till instance is ready
waiter = ec2_client.get_waiter('instance_running')
waiter.wait(InstanceIds=instances_ids)


In [49]:
## create new EIP and attach it to existing EC2 instance
instance_id = instances[0].instance_id
try:
    allocation = ec2_client.allocate_address(Domain='vpc')
    response = ec2_client.associate_address(AllocationId=allocation['AllocationId'],
                                     InstanceId=instance_id)
    print(response)
except ClientError as e:
    print(e)
print(f"\nThe EIP is: {allocation['PublicIp']}")

{'AssociationId': 'eipassoc-0d7a22c90038871d8', 'ResponseMetadata': {'RequestId': 'e8bc7259-a4e7-4f58-acc2-3d5bbdff9bd5', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'e8bc7259-a4e7-4f58-acc2-3d5bbdff9bd5', 'content-type': 'text/xml;charset=UTF-8', 'content-length': '295', 'date': 'Tue, 06 Oct 2020 19:32:33 GMT', 'server': 'AmazonEC2'}, 'RetryAttempts': 0}}

The EIP is: 3.18.83.229


In [51]:
## 3.1 create security group for RDS
rds_sg = ec2_client.create_security_group(
#      VpcId=vpc.id,
    Description="IMMIGRATE_RDS_SG",
    GroupName="IMMIGRATE_RDS_SG",
    VpcId=vpc.id,
    TagSpecifications=[{
        'ResourceType':'security-group',
         'Tags':[{"Key": "Name", "Value": RDS_SG_NAME}]
        }]
)

In [52]:
rds_sg

{'GroupId': 'sg-0040e35bef1440512',
 'Tags': [{'Key': 'Name', 'Value': 'IMMIGRATE_RDS_SG'}],
 'ResponseMetadata': {'RequestId': '84f53a48-62cc-418e-b2fd-7395633d92d8',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '84f53a48-62cc-418e-b2fd-7395633d92d8',
   'content-type': 'text/xml;charset=UTF-8',
   'content-length': '413',
   'date': 'Tue, 06 Oct 2020 19:35:53 GMT',
   'server': 'AmazonEC2'},
  'RetryAttempts': 0}}

In [53]:
## 3.2 get default security group id
sg = ec2_client.describe_security_groups(Filters=[{'Name': 'vpc-id',
                                                    'Values': [vpc.vpc_id]}
                                                 ])

for sg_ in sg['SecurityGroups']:
    if sg_['Description'] == 'default VPC security group':
        sg_default = sg_
        break
        
sg_default_id = sg_default['GroupId']

In [54]:
## 3.3 allow TCP connect to port 5432
ec2_client.authorize_security_group_ingress(GroupId = rds_sg['GroupId'],
                                        IpPermissions=[
                                        {
                                            'FromPort': 5432,
                                            'IpProtocol': 'tcp',
                                            'UserIdGroupPairs': [
                                                {
                                                    'GroupId': sg_default_id,
                                                    'Description': 'Postgres RDS',
                                                },
                                            ],
                                            'ToPort': 5432,
                                        },
                                    ],)

{'ResponseMetadata': {'RequestId': 'dc9431b0-a659-46c8-aa21-c73f982dc699',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'dc9431b0-a659-46c8-aa21-c73f982dc699',
   'content-type': 'text/xml;charset=UTF-8',
   'content-length': '259',
   'date': 'Tue, 06 Oct 2020 19:37:54 GMT',
   'server': 'AmazonEC2'},
  'RetryAttempts': 0}}

In [57]:
## 3.4 create subnet group
subnet_group = rds_client.create_db_subnet_group(
    DBSubnetGroupName='IMMIGRATE_RDS_SUBNET_GROUPS',
    DBSubnetGroupDescription='IMMIGRATE_RDS_SUBNET_GROUPS',
    SubnetIds=[
        subnet_prv_empty.id,
        subnet_prv2_rds.id
    ],
    Tags=[
        {
            'Key': 'Name',
            'Value': 'IMMIGRATE_RDS_SUBNET_GROUPS'
        },
    ]
)

{'DBSubnetGroup': {'DBSubnetGroupName': 'immigrate_rds_subnet_groups',
  'DBSubnetGroupDescription': 'IMMIGRATE_RDS_SUBNET_GROUPS',
  'VpcId': 'vpc-0808bc58b136772dd',
  'SubnetGroupStatus': 'Complete',
  'Subnets': [{'SubnetIdentifier': 'subnet-02a1ee5e6d38a029f',
    'SubnetAvailabilityZone': {'Name': 'us-east-2b'},
    'SubnetOutpost': {},
    'SubnetStatus': 'Active'},
   {'SubnetIdentifier': 'subnet-074f7c1568b4e6cad',
    'SubnetAvailabilityZone': {'Name': 'us-east-2c'},
    'SubnetOutpost': {},
    'SubnetStatus': 'Active'}],
  'DBSubnetGroupArn': 'arn:aws:rds:us-east-2:736387989270:subgrp:immigrate_rds_subnet_groups'},
 'ResponseMetadata': {'RequestId': 'd48ec3de-d14e-4c58-858b-6541947f3332',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'd48ec3de-d14e-4c58-858b-6541947f3332',
   'content-type': 'text/xml',
   'content-length': '1296',
   'date': 'Tue, 06 Oct 2020 19:39:56 GMT'},
  'RetryAttempts': 0}}

In [62]:
RDS_NAME

'IMMIGRATE_DEMOGRAPHICS_DB'

In [70]:
## 3.5 create database instance
rds_instance = rds_client.create_db_instance(
                   DBInstanceIdentifier=RDS_IDENTIFIER,
                   AllocatedStorage=20,
                   DBName=RDS_NAME,
                   Engine='postgres',
                   # General purpose SSD
                   StorageType='gp2',
#                    StorageEncrypted=True,
#                    AutoMinorVersionUpgrade=True,
                   # Set this to true later?
                   MultiAZ=False,
                   MasterUsername='immigrate_demo',
                   MasterUserPassword='=p<%Gs7TS_pd%-',
                   VpcSecurityGroupIds=[rds_sg['GroupId']],
                   DBSubnetGroupName='immigrate_rds_subnet_groups',
                   DBInstanceClass='db.t2.micro' 
                        #,
#                    Tags=[{'Key': 'MyTag', 'Value': 'Hawaii'}]
)

# wait till instance is ready
waiter = rds_client.get_waiter('db_instance_available')
waiter.wait(DBInstanceIdentifier=RDS_IDENTIFIER)

In [71]:
rds_instance

{'DBInstance': {'DBInstanceIdentifier': 'immigrate-demo-db',
  'DBInstanceClass': 'db.t2.micro',
  'Engine': 'postgres',
  'DBInstanceStatus': 'creating',
  'MasterUsername': 'immigrate_demo',
  'DBName': 'IMMIGRATE_DEMOGRAPHICS_DB',
  'AllocatedStorage': 20,
  'PreferredBackupWindow': '08:50-09:20',
  'BackupRetentionPeriod': 1,
  'DBSecurityGroups': [],
  'VpcSecurityGroups': [{'VpcSecurityGroupId': 'sg-0040e35bef1440512',
    'Status': 'active'}],
  'DBParameterGroups': [{'DBParameterGroupName': 'default.postgres12',
    'ParameterApplyStatus': 'in-sync'}],
  'DBSubnetGroup': {'DBSubnetGroupName': 'immigrate_rds_subnet_groups',
   'DBSubnetGroupDescription': 'IMMIGRATE_RDS_SUBNET_GROUPS',
   'VpcId': 'vpc-0808bc58b136772dd',
   'SubnetGroupStatus': 'Complete',
   'Subnets': [{'SubnetIdentifier': 'subnet-02a1ee5e6d38a029f',
     'SubnetAvailabilityZone': {'Name': 'us-east-2b'},
     'SubnetOutpost': {},
     'SubnetStatus': 'Active'},
    {'SubnetIdentifier': 'subnet-074f7c1568b4e6ca